In [1]:
# Eval data link: https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/447dd4/dataset_eval.zip
!curl -L -o ../data/dataset_eval.zip https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/447dd4/dataset_eval.zip
!unzip -d ../data/eval ../data/dataset_eval.zip
!rm ../data/dataset_eval.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  134k  100  134k    0     0   170k      0 --:--:-- --:--:-- --:--:--  170k
Archive:  ../data/dataset_eval.zip
  inflating: ../data/eval/answers.json  
  inflating: ../data/eval/corpus.json  
  inflating: ../data/eval/errors.json  
  inflating: ../data/eval/queries.json  
  inflating: ../data/eval/relevant_docs.json  


# Create database

In [ ]:
import sys
sys.path.append("../src")

from rag_alm_assistant.ingestion import full_ingestion_pipeline
from rag_alm_assistant.constants import DIC_DIR, VECTORSTORE_DIR, EMBEDDING_MODEL_NAME

vector_store, raw_docs, chunks = full_ingestion_pipeline(
    dic_dir="../data/DIC",
    persist_directory="../data/vector_store",
    model_name=EMBEDDING_MODEL_NAME,
)

print("Nb docs bruts :", len(raw_docs))
print("Nb chunks     :", len(chunks))
print("Vector store  :", VECTORSTORE_DIR)

# Health check

In [ ]:
# Check how ert score works
from bert_score import score

# 1) même phrase => score proche de 1
refs = ["L’OPCVM est un fonds d’investissement collectif en valeurs mobilières."]
preds = ["L’OPCVM est un fonds d’investissement collectif en valeurs mobilières."]

P, R, F1 = score(preds, refs, lang="fr", rescale_with_baseline=True)
print("Identiques F1:", F1.mean().item())

# 2) phrases proches => score élevé
refs2 = ["L’OPCVM permet un investissement collectif sur les marchés financiers."]
preds2 = ["Un OPCVM est un produit qui permet d’investir collectivement en titres financiers."]

P2, R2, F12 = score(preds2, refs2, lang="fr", rescale_with_baseline=True)
print("Semblables F1:", F12.mean().item())

In [1]:
## Check few samples of answers:
import json
from pathlib import Path
import sys
sys.path.append("../src")
from rag_alm_assistant.orchestrator import RAGOrchestrator
import random

EVAL_DIR = Path("../data/eval")
with open(EVAL_DIR / "queries.json", "r", encoding="utf-8") as f:
    queries = json.load(f)
with open(EVAL_DIR / "answers.json", "r", encoding="utf-8") as f:
    gold_answers = json.load(f)

orchestrator = RAGOrchestrator(use_reranker = True, use_memory= False)

sample_uuids = random.sample(list(queries.keys()), 5)

for uid in sample_uuids:
    q = queries[uid]
    ref_answer = gold_answers[uid]
    pred_answer, pred_sources = orchestrator.ask(q)

    print("==== UID:", uid, "====")
    print("Question :", q)
    print("\nGold answer :\n", ref_answer)
    print("\nPred answer :\n", pred_answer)
    print("\nSources prédictes :", pred_sources)
    print("\n" + "="*80 + "\n")

/workspace/RAG_ALM_assistant/notebooks/../src/rag_alm_assistant/retrieval/retriever.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, encode_kwargs=encode_kwargs)


[2025-11-14 10:54:38] [INFO] [rag_alm_assistant.ingestion] [INGESTION] Loading DIC documents from: /workspace/RAG_ALM_assistant/data/DIC
[2025-11-14 10:54:54] [INFO] [rag_alm_assistant.ingestion] [WARNING] Error while loading '/workspace/RAG_ALM_assistant/data/DIC/Swisslife.pdf': cryptography>=3.1 is required for AES algorithm
[2025-11-14 10:54:54] [INFO] [rag_alm_assistant.ingestion] 173 documents processed.
[2025-11-14 10:54:54] [INFO] [rag_alm_assistant.ingestion] [INGESTION] Chunking documents...
[2025-11-14 10:54:54] [INFO] [rag_alm_assistant.ingestion] 571 chunks created from 173 documents.
[2025-11-14 10:54:54] [INFO] [rag_alm_assistant.ingestion] [INGESTION] Building vector store...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


[2025-11-14 10:55:10] [INFO] [rag_alm_assistant.ingestion] Vector store built and persisted at: /workspace/RAG_ALM_assistant/data/vector_store


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[2025-11-14 10:55:26] [INFO] [rag_alm_assistant.orchestrator] Generate answer without memory for question: Qu'est-ce qui est investi et maintenu pour chaque Compartiment d’Amundi Index Solutions en cas d'incapacité d'Amundi Luxembourg S.A. à effectuer les versements ?


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


==== UID: 68773308-ef6a-4c53-9c51-1ab2336dfa3f ====
Question : Qu'est-ce qui est investi et maintenu pour chaque Compartiment d’Amundi Index Solutions en cas d'incapacité d'Amundi Luxembourg S.A. à effectuer les versements ?

Gold answer :
 Un pool d’actifs distinct

Pred answer :
 Les revenus sont réinvestis automatiquement.

Sources prédictes : [{'dic_name': 'KIDPRIIPs_391793_48169_FRA_FRA_20231030.pdf', 'page': 1}]


[2025-11-14 10:55:28] [INFO] [rag_alm_assistant.orchestrator] Generate answer without memory for question: Quels risques supplémentaires sont mentionnés dans le document ?
==== UID: 22596d2d-f99c-40e2-8db1-436dbffa8c31 ====
Question : Quels risques supplémentaires sont mentionnés dans le document ?

Gold answer :
 Le risque de liquidité du marché peut accentuer la variation des performances du produit. Ce produit ne prévoyant pas de protection contre les aléas de marché, vous pourriez perdre tout ou partie de votre investissement.

Pred answer :
 Le risque de défaillanc

# Evaluation without reranker

In [ ]:
import json
from pathlib import Path
import sys
sys.path.append("../src")
from rag_alm_assistant.orchestrator import RAGOrchestrator

from bert_score import score

EVAL_DIR = Path("../data/eval")

with open(EVAL_DIR / "queries.json", "r", encoding="utf-8") as f:
    queries = json.load(f)

with open(EVAL_DIR / "answers.json", "r", encoding="utf-8") as f:
    gold_answers = json.load(f)

with open(EVAL_DIR / "relevant_docs.json", "r", encoding="utf-8") as f:
    gold_sources = json.load(f)

uuids = list(queries.keys())
orchestrator = RAGOrchestrator(use_reranker = False, use_memory= False)

preds = []  # answer from our pipeline
refs = []   # references

for uid in uuids:
    q = queries[uid]
    ref_answer = gold_answers[uid]
    ref_sources = gold_sources[uid]

    pred_answer, pred_source = orchestrator.ask(q)
    # print("\nSources prédictes :", pred_source)

    preds.append(pred_answer)
    refs.append(ref_answer)

In [3]:
# Calcul BERTScore
P, R, F1 = score(
    cands=preds,
    refs=refs,
    lang = "fr",
    rescale_with_baseline=True
)

f1_mean = F1.mean().item()
print(f"Mean BERTScore F1: {f1_mean:.4f}  ({f1_mean*100:.2f}%)")

NameError: name 'score' is not defined

# Evaluation with reranker

In [ ]:
import json
from pathlib import Path
import sys
sys.path.append("../src")
from rag_alm_assistant.orchestrator import RAGOrchestrator

from bert_score import score

EVAL_DIR = Path("../data/eval")

with open(EVAL_DIR / "queries.json", "r", encoding="utf-8") as f:
    queries = json.load(f)

with open(EVAL_DIR / "answers.json", "r", encoding="utf-8") as f:
    gold_answers = json.load(f)

with open(EVAL_DIR / "relevant_docs.json", "r", encoding="utf-8") as f:
    gold_sources = json.load(f)

uuids = list(queries.keys())
orchestrator = RAGOrchestrator(use_reranker = True, use_memory=False)

preds = []  # answer from our pipeline
refs = []   # references

for uid in uuids:
    q = queries[uid]
    ref_answer = gold_answers[uid]
    ref_sources = gold_sources[uid]

    pred_answer, pred_source = orchestrator.ask(q)

    preds.append(pred_answer)
    refs.append(ref_answer)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


[2025-11-14 10:57:11] [INFO] [rag_alm_assistant.orchestrator] Generate answer without memory for question: Quel est l'objectif de gestion du FCP décrit dans le document ?
[2025-11-14 10:57:15] [INFO] [rag_alm_assistant.orchestrator] Generate answer without memory for question: Quels critères sont pris en compte dans les décisions d'investissement du FCP ?
[2025-11-14 10:57:19] [INFO] [rag_alm_assistant.orchestrator] Generate answer without memory for question: A quoi vise la prise en compte des critères ESG dans la sélection de titres du FCP ?
[2025-11-14 10:57:22] [INFO] [rag_alm_assistant.orchestrator] Generate answer without memory for question: Quels sont les scénarios mentionnés pour un investissement utilisant un mandataire approprié ?
[2025-11-14 10:57:25] [INFO] [rag_alm_assistant.orchestrator] Generate answer without memory for question: Quelles sont les dates associées à chaque scénario pour un investissement utilisant un mandataire approprié ?
[2025-11-14 10:57:31] [INFO] [r

In [3]:
# Calcul BERTScore
P, R, F1 = score(
    cands=preds,
    refs=refs,
    lang = "fr",
    rescale_with_baseline=True
)

f1_mean = F1.mean().item()
print(f"Mean BERTScore F1: {f1_mean:.4f}  ({f1_mean*100:.2f}%)")

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Mean BERTScore F1: 0.3519  (35.19%)
